In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import calendar

from datetime import datetime, date
from numpy import linalg as la
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

#pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 10000)

In [2]:
df = pd.read_csv("Historical_Product_Demand.csv")
df.head(5)

,Product_Code,Warehouse,Product_Category,Date,Order_Demand
0,Product_0993,Whse_J,Category_028,2012/7/27,100
1,Product_0979,Whse_J,Category_028,2012/1/19,500
2,Product_0979,Whse_J,Category_028,2012/2/3,500
3,Product_0979,Whse_J,Category_028,2012/2/9,500
4,Product_0979,Whse_J,Category_028,2012/3/2,500


## Check for NaNs in columns and drop those rows

In [3]:
print (df.isna().sum())
df.dropna(axis=0, inplace=True) #remove all rows with na's.
df.reset_index(drop=True)
print (df.isna().sum())

Product_Code            0
Warehouse               0
Product_Category        0
Date                11239
Order_Demand            0
dtype: int64
Product_Code        0
Warehouse           0
Product_Category    0
Date                0
Order_Demand        0
dtype: int64


# Circular encoding of month

In [4]:
# Convert order demand column to int
def str2num(val):
    num = re.findall(r'\d+', val)
    return int(''.join(num))

def month_year_extract(date_str):
    date_obj = datetime.strptime(date_str, '%Y/%m/%d')
    return date(date_obj.year, date_obj.month, 1)

def month_nr_extract(date_str):
    date_obj = datetime.strptime(date_str, '%Y/%m/%d')
    return date_obj.month

def month_name_extract(date_str):
    date_obj = datetime.strptime(date_str, '%Y/%m/%d')
    return calendar.month_name[date_obj.month]
    
def year_extract(date_str):
    date_obj = datetime.strptime(date_str, '%Y/%m/%d')
    return date_obj.year

    
def month_circular_encoding_sin(date_str):
    date_obj = datetime.strptime(date_str, '%Y/%m/%d')
    return np.sin(2 * np.pi * float(date_obj.month)/12)

def month_circular_encoding_cos(date_str):
    date_obj = datetime.strptime(date_str, '%Y/%m/%d')
    return np.cos(2 * np.pi * float(date_obj.month)/12)

In [5]:
df.loc[:,"Order_Demand"] = df.loc[:,"Order_Demand"].apply(str2num)
df = df.astype({'Order_Demand': 'int32'},copy=True)

df["month_year"] = df["Date"].apply(month_year_extract)
df["month_nr"] = df["Date"].apply(month_nr_extract)
df["month"] = df["Date"].apply(month_name_extract)
df["year"] = df["Date"].apply(year_extract)
df["month_sin"] = df["Date"].apply(month_circular_encoding_sin)
df["month_cos"] = df["Date"].apply(month_circular_encoding_cos)
df.drop(columns=['Date'], inplace=True)

display(df.info())
display(df.head(5))
display(df.nunique())

<class 'pandas.core.frame.DataFrame'>
Index: 1037336 entries, 0 to 1048574
Data columns (total 10 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   Product_Code      1037336 non-null  object 
 1   Warehouse         1037336 non-null  object 
 2   Product_Category  1037336 non-null  object 
 3   Order_Demand      1037336 non-null  int32  
 4   month_year        1037336 non-null  object 
 5   month_nr          1037336 non-null  int64  
 6   month             1037336 non-null  object 
 7   year              1037336 non-null  int64  
 8   month_sin         1037336 non-null  float64
 9   month_cos         1037336 non-null  float64
dtypes: float64(2), int32(1), int64(2), object(5)
memory usage: 83.1+ MB


None

,Product_Code,Warehouse,Product_Category,Order_Demand,month_year,month_nr,month,year,month_sin,month_cos
0,Product_0993,Whse_J,Category_028,100,2012-07-01,7,July,2012,-0.500000,-8.660254e-01
1,Product_0979,Whse_J,Category_028,500,2012-01-01,1,January,2012,0.500000,8.660254e-01
2,Product_0979,Whse_J,Category_028,500,2012-02-01,2,February,2012,0.866025,5.000000e-01
3,Product_0979,Whse_J,Category_028,500,2012-02-01,2,February,2012,0.866025,5.000000e-01
4,Product_0979,Whse_J,Category_028,500,2012-03-01,3,March,2012,1.000000,6.123234e-17


Product_Code        2160
Warehouse              4
Product_Category      33
Order_Demand        3309
month_year            68
month_nr              12
month                 12
year                   7
month_sin             11
month_cos             11
dtype: int64

In [6]:
#warehouse_df = pd.get_dummies(df[['Warehouse']]).astype('int32')
#new_df = pd.concat([warehouse_df, df],axis=1)
#display(print (df.isna().sum()))
#new_df

## Predict based on Product Category

In [7]:
product_categories = df["Product_Category"].unique()
display(product_categories)

array(['Category_028', 'Category_006', 'Category_001', 'Category_019',
       'Category_005', 'Category_030', 'Category_032', 'Category_011',
       'Category_033', 'Category_007', 'Category_015', 'Category_021',
       'Category_024', 'Category_026', 'Category_023', 'Category_022',
       'Category_009', 'Category_003', 'Category_004', 'Category_018',
       'Category_020', 'Category_013', 'Category_008', 'Category_017',
       'Category_031', 'Category_025', 'Category_010', 'Category_029',
       'Category_012', 'Category_002', 'Category_014', 'Category_027',
       'Category_016'], dtype=object)

In [30]:
# Loop through product categories and train a regression model for each category
data_df = df[['Order_Demand','month_sin','month_cos']]
agg_dict = {'month_sin':['mean'], 'month_cos':['mean'], 'Order_Demand':['sum']}
agg_dict = {'month_nr':['mean'], 'month_sin':['mean'], 'month_cos':['mean'], 'Order_Demand':['sum']}
models_ls = []

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

for category in product_categories:
    category_df = df[ (df['Product_Category'] == category) & (df['year'] > 2011) & (df['year'] < 2017)]
    
    #grouped_data = category_df[['year', 'month_nr', 'month_sin', 'month_cos','Order_Demand']].sort_values(['month_nr'],ascending=True).groupby(['year', 'month_nr']).agg(agg_dict)
    grouped_data = category_df[['year', 'month', 'month_nr', 'month_sin', 'month_cos','Order_Demand']].sort_values(['month_nr'],ascending=True).groupby(['year', 'month']).agg(agg_dict)
    
    X = grouped_data[['month_sin', 'month_cos']]
    y = grouped_data[['Order_Demand']].values.ravel()

    X = StandardScaler().fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

    #display(grouped_data)
    #display(X)

    dectree_r2  =0
    randfor_r2 = 0
    
    model = DecisionTreeRegressor(criterion='squared_error', random_state=0).fit(X_train, y_train)
    #y_predicted = model.predict(X_test)
    dectree_r2 = model.score(X_test, y_test)

    model = RandomForestRegressor(n_estimators = 100, criterion="squared_error", random_state=0).fit(X_train, y_train)
    #y_predicted = model.predict(X_test)
    randfor_r2 = model.score(X_test, y_test)

    
    print(f"{category}, DecisionTree R2 = {dectree_r2}")
    print(f"{category}, RandomForest R2 = {randfor_r2}")
    #break

Category_028, DecisionTree R2 = -2.6765575457072295
Category_028, RandomForest R2 = -3.3337787368382736
Category_006, DecisionTree R2 = -0.12606111499990869
Category_006, RandomForest R2 = -0.10261037190686917
Category_001, DecisionTree R2 = -0.5361040788187081
Category_001, RandomForest R2 = -0.4715510685300357
Category_019, DecisionTree R2 = -0.119778825417022
Category_019, RandomForest R2 = -0.0879264864623297
Category_005, DecisionTree R2 = -0.1606523897496881
Category_005, RandomForest R2 = -0.18234976467215858
Category_030, DecisionTree R2 = -0.8655863597311979
Category_030, RandomForest R2 = -0.870060107406194
Category_032, DecisionTree R2 = -0.17113151133913163
Category_032, RandomForest R2 = -0.16989584167362026
Category_011, DecisionTree R2 = -0.3924045272671006
Category_011, RandomForest R2 = -0.3472286956637254
Category_033, DecisionTree R2 = -1.7483955659276549
Category_033, RandomForest R2 = -1.645739076137028
Category_007, DecisionTree R2 = -0.7394712072712095
Category_0